In [1]:
import igraph as ig


track_network_path = "../network-parser/output/track_graph.graphml"
artist_network_path = "../network-parser/output/artist_graph.graphml"

# read graphml files
track_network = ig.Graph.Read_GraphML(track_network_path)
# artist_network = ig.Graph.Read_GraphML(artist_network_path)

# force convert to undirected
track_network.to_undirected()
# artist_network.to_undirected()

# force convert to simple
# track_network.simplify()
# artist_network.simplify()

# # confirm
# print(track_network.is_simple())
# print(artist_network.is_simple())

graph = track_network



# Plain Random Walk

In [18]:
import igraph as ig
import random
import json
from tqdm.notebook import tqdm
import numpy as np
import logging
logger = logging.getLogger(__name__)


# setup logging to print all info
logging.basicConfig(level=logging.INFO)

# Step 1: Load the graph from GraphML
# graph = ig.Graph.Read_GraphML(track_network_path)
# graph.to_undirected()  # Ensure the graph is undirected

random.seed(42)

# Step 2: Random walk function avoiding tracks in user's playlist
def weighted_random_walk_exclude_user_pid(graph, start_node, user_pid, num_steps):
    walk = [start_node]
    current_node = start_node

    for _ in range(num_steps):
        neighbors = graph.neighbors(current_node, mode="all")

        # Exclude neighbors associated with the user's playlist ID
        valid_neighbors = [
            neighbor for neighbor in neighbors
            #if user_pid not in map(int, graph.vs[neighbor]["playlists"].split(","))
        ]
        if not valid_neighbors:
            break  # Stop if no valid neighbors

        edge_ids = [graph.get_eid(current_node, neighbor) for neighbor in valid_neighbors]
        weights = [graph.es[edge_id]["weight"] for edge_id in edge_ids]

        next_node = random.choices(valid_neighbors, weights=weights, k=1)[0]
        walk.append(next_node)
        current_node = next_node

    return walk

# Step 3: Recommendation function
def recommend_tracks_by_pid(graph, user_pid, user_track, num_steps=5, top_n=1):
    try:
        start_node = graph.vs.find(id=user_track).index
    except ValueError:
        # raise ValueError(f"Track '{user_track}' not found in the graph.")
        # print('nope')
        return None

    walk = weighted_random_walk_exclude_user_pid(graph, start_node, user_pid, num_steps)

    visit_counts = {}
    for node in walk:
        track_id = graph.vs[node]["id"]
        #if user_pid not in map(int, graph.vs[node]["playlists"].split(",")):
        visit_counts[track_id] = visit_counts.get(track_id, 0) + 1

    recommended_tracks = sorted(visit_counts, key=visit_counts.get, reverse=True)
    eligible_tracks = [track for track in recommended_tracks if track != user_track]
    return eligible_tracks[:top_n]

# Step 4: Accuracy testing function
def test_recommendation_accuracy(graph, test_data_path, num_steps=5, top_n=1):
    with open(test_data_path, "r") as f:
        test_data = json.load(f)

    playlists = test_data["playlists"]

    input_track_uris_per_playlist = {}
    eligible_input_uri_per_playlist = {}
    for playlist in tqdm(playlists, desc="Processing playlists"):
        this_track_uris = [track["track_uri"] for track in playlist["tracks"]]
        input_track_uris_per_playlist[playlist["pid"]] = this_track_uris
        tracks_in_graph = [track_uri for track_uri in this_track_uris if track_uri in graph.vs["id"]]
        eligible_input_uri_per_playlist[playlist["pid"]] = random.choice(tracks_in_graph) if len(tracks_in_graph) > 0 else None

    # filter key
    testable_playlists = [pid for pid in input_track_uris_per_playlist.keys() if eligible_input_uri_per_playlist[pid] != None]
    recommendation_success = np.zeros(len(testable_playlists))
    recommended_degree = np.zeros(len(testable_playlists))

    for i, pid in enumerate(tqdm(testable_playlists, desc="Testing playlists")):
        input_track_uris = input_track_uris_per_playlist[pid]
        eligible_input_uri = eligible_input_uri_per_playlist[pid]
        recommended_tracks = recommend_tracks_by_pid(graph, pid, eligible_input_uri, num_steps, top_n)
        if recommended_tracks is not None:
            recommended_degrees = [graph.degree(graph.vs.find(id=track).index) for track in recommended_tracks]
            recommended_degree[i] = np.mean(recommended_degrees)
            if any(track in recommended_tracks for track in input_track_uris):
                recommendation_success[i] = 1
        else:
            recommended_degree[i] = 0


    accuracy = np.mean(recommendation_success)
    print(f"Accuracy: {accuracy * 100:.2f}%")

    mean_recommended_degree = np.mean(recommended_degree)
    print(f"Mean recommended degree: {mean_recommended_degree:.2f}")

    # recommendation_scores = recommendation_success / np.log10(recommended_degree)
    recommendation_scores = recommendation_success / recommended_degree
    mean_recommendation_score = np.mean(recommendation_scores)
    print(f"Mean recommendation score: {mean_recommendation_score:.3e}")

    return accuracy

# Step 5: Run the test
test_data_path = "../dataset/data/mpd.slice.999000-999999.json"  # Path to your test JSON file
test_recommendation_accuracy(graph, test_data_path, num_steps=7, top_n=1)


Processing playlists:   0%|          | 0/1000 [00:00<?, ?it/s]

Testing playlists:   0%|          | 0/987 [00:00<?, ?it/s]

Accuracy: 2.63%
Mean recommended degree: 785.40
Mean recommendation score: 2.336e-04


0.02634245187436677

# Node2Vec Random Walk

In [19]:
import igraph as ig
import random
import json

# Step 1: Load the graph from GraphML
# graph = ig.Graph.Read_GraphML("../network-parser/track_graph.graphml")
# graph.to_undirected()  # Ensure the graph is undirected

# random.seed(43)

# Step 2: Define the Node2Vec random walk function
def node2vec_random_walk_exclude_user_pid(graph, start_node, user_pid, num_steps, p=1.0, q=1.0):
    """
    Perform a Node2Vec-style random walk while avoiding tracks in the user's playlist ID.
    
    Args:
        graph (igraph.Graph): The input graph with edge weights.
        start_node (int): The starting node index for the random walk.
        user_pid (int): The user's playlist ID to exclude tracks from.
        num_steps (int): Number of steps for the random walk.
        p (float): Return parameter (controls likelihood of returning to the previous node).
        q (float): In-out parameter (controls exploration of closer vs. farther nodes).
        
    Returns:
        list: A list of visited node indices during the random walk.
    """
    walk = [start_node]
    current_node = start_node
    previous_node = None

    for _ in range(num_steps):
        neighbors = graph.neighbors(current_node, mode="all")

        # Exclude neighbors associated with the user's playlist ID
        valid_neighbors = [
            neighbor for neighbor in neighbors
            #if user_pid not in map(int, graph.vs[neighbor]["playlists"].split(","))
        ]
        if not valid_neighbors:
            break  # Stop if no valid neighbors

        edge_ids = [graph.get_eid(current_node, neighbor) for neighbor in valid_neighbors]
        weights = []

        for neighbor, edge_id in zip(valid_neighbors, edge_ids):
            if previous_node is not None and neighbor == previous_node:
                # Bias for returning to the previous node
                weights.append(graph.es[edge_id]["weight"] / p)
            elif previous_node is not None:
                # Bias for exploring farther or closer nodes
                distance_factor = 1.0 if neighbor in graph.neighbors(previous_node) else q
                weights.append(graph.es[edge_id]["weight"] / distance_factor)
            else:
                # First step, no previous node
                weights.append(graph.es[edge_id]["weight"])

        next_node = random.choices(valid_neighbors, weights=weights, k=1)[0]
        walk.append(next_node)
        previous_node = current_node
        current_node = next_node

    return walk

# Step 3: Recommendation function
def recommend_tracks_by_pid_node2vec(graph, user_pid, user_track, num_steps=5, top_n=1, p=1.0, q=1.0):
    """
    Recommend tracks based on a Node2Vec random walk avoiding tracks in the user's playlist.
    
    Args:
        graph (igraph.Graph): The input graph with edge weights.
        user_pid (int): The user's playlist ID to exclude tracks from.
        user_track (str): The track ID of the starting node.
        num_steps (int): Number of steps for the random walk.
        top_n (int): Number of recommendations to return.
        p (float): Return parameter (controls likelihood of returning to the previous node).
        q (float): In-out parameter (controls exploration of closer vs. farther nodes).
        
    Returns:
        list: A list of recommended track IDs.
    """
    try:
        start_node = graph.vs.find(id=user_track).index
    except ValueError:
        #raise ValueError(f"Track '{user_track}' not found in the graph.")
        return None

    walk = node2vec_random_walk_exclude_user_pid(graph, start_node, user_pid, num_steps, p, q)

    visit_counts = {}
    for node in walk:
        track_id = graph.vs[node]["id"]
        #if user_pid not in map(int, graph.vs[node]["playlists"].split(",")):
        visit_counts[track_id] = visit_counts.get(track_id, 0) + 1

    recommended_tracks = sorted(visit_counts, key=visit_counts.get, reverse=True)
    eligible_tracks = [track for track in recommended_tracks if track != user_track]
    return eligible_tracks[:top_n]

# Step 4: Accuracy testing function
def test_recommendation_accuracy_node2vec(graph, test_data_path, num_steps=5, top_n=1, p=1.0, q=1.0):
    with open(test_data_path, "r") as f:
        test_data = json.load(f)

    playlists = test_data["playlists"]
    
    input_track_uris_per_playlist = {}
    eligible_input_uri_per_playlist = {}
    for playlist in tqdm(playlists, desc="Processing playlists"):
        this_track_uris = [track["track_uri"] for track in playlist["tracks"]]
        input_track_uris_per_playlist[playlist["pid"]] = this_track_uris
        tracks_in_graph = [track_uri for track_uri in this_track_uris if track_uri in graph.vs["id"]]
        eligible_input_uri_per_playlist[playlist["pid"]] = random.choice(tracks_in_graph) if len(tracks_in_graph) > 0 else None

    # filter key
    testable_playlists = [pid for pid in input_track_uris_per_playlist.keys() if eligible_input_uri_per_playlist[pid] != None]
    recommendation_success = np.zeros(len(testable_playlists))
    recommended_degree = np.zeros(len(testable_playlists))

    for i, pid in enumerate(tqdm(testable_playlists, desc="Testing playlists")):
        input_track_uris = input_track_uris_per_playlist[pid]
        eligible_input_uri = eligible_input_uri_per_playlist[pid]
        recommended_tracks = recommend_tracks_by_pid_node2vec(graph, pid, eligible_input_uri, num_steps, top_n, p, q)
        if recommended_tracks is not None:
            recommended_degrees = [graph.degree(graph.vs.find(id=track).index) for track in recommended_tracks]
            recommended_degree[i] = np.mean(recommended_degrees)
            if any(track in recommended_tracks for track in input_track_uris):
                recommendation_success[i] = 1
        else:
            recommended_degree[i] = 0

    accuracy = np.mean(recommendation_success)
    print(f"Accuracy: {accuracy * 100:.2f}%")

    mean_recommended_degree = np.mean(recommended_degree)
    print(f"Mean recommended degree: {mean_recommended_degree:.2f}")

    # recommendation_scores = recommendation_success / np.log10(recommended_degree)
    recommendation_scores = recommendation_success / recommended_degree
    mean_recommendation_score = np.mean(recommendation_scores)
    print(f"Mean recommendation score: {mean_recommendation_score:.3e}")

    return accuracy


# Step 5: Run the test
test_data_path = "../dataset/data/mpd.slice.999000-999999.json"  # Path to your test JSON file
test_recommendation_accuracy_node2vec(graph, test_data_path, p=2.0, q=2.0, num_steps=7, top_n=1)


Processing playlists:   0%|          | 0/1000 [00:00<?, ?it/s]

Testing playlists:   0%|          | 0/987 [00:00<?, ?it/s]

Accuracy: 3.24%
Mean recommended degree: 830.98
Mean recommendation score: 1.286e-04


0.03242147922998987

# Restart Random Walk

In [20]:
import igraph as ig
import random
import json

# Step 1: Load the graph from GraphML
# graph = ig.Graph.Read_GraphML("../network-parser/track_graph.graphml")
# graph.to_undirected()  # Ensure the graph is undirected

# random.seed(44)

# Step 2: Define the Restart Random Walk function
def restart_random_walk_exclude_user_pid(graph, start_node, user_pid, num_steps, restart_prob=0.15):
    """
    Perform a restart random walk while avoiding tracks in the user's playlist ID.
    
    Args:
        graph (igraph.Graph): The input graph with edge weights.
        start_node (int): The starting node index for the random walk.
        user_pid (int): The user's playlist ID to exclude tracks from.
        num_steps (int): Number of steps for the random walk.
        restart_prob (float): Probability of restarting the walk at the starting node.
        
    Returns:
        list: A list of visited node indices during the random walk.
    """
    walk = [start_node]
    current_node = start_node

    for _ in range(num_steps):
        if random.random() < restart_prob:
            # Restart to the starting node
            current_node = start_node
        else:
            neighbors = graph.neighbors(current_node, mode="all")

            # Exclude neighbors associated with the user's playlist ID
            valid_neighbors = [
                neighbor for neighbor in neighbors
                #if user_pid not in map(int, graph.vs[neighbor]["playlists"].split(","))
            ]
            if not valid_neighbors:
                break  # Stop if no valid neighbors

            # Get edge weights for valid neighbors
            edge_ids = [graph.get_eid(current_node, neighbor) for neighbor in valid_neighbors]
            weights = [graph.es[edge_id]["weight"] for edge_id in edge_ids]

            current_node = random.choices(valid_neighbors, weights=weights, k=1)[0]

        walk.append(current_node)

    return walk

# Step 3: Recommendation function
def recommend_tracks_by_pid_restart(graph, user_pid, user_track, num_steps=5, top_n=1, restart_prob=0.15):
    """
    Recommend tracks based on a restart random walk avoiding tracks in the user's playlist.
    
    Args:
        graph (igraph.Graph): The input graph with edge weights.
        user_pid (int): The user's playlist ID to exclude tracks from.
        user_track (str): The track ID of the starting node.
        num_steps (int): Number of steps for the random walk.
        top_n (int): Number of recommendations to return.
        restart_prob (float): Probability of restarting the walk at the starting node.
        
    Returns:
        list: A list of recommended track IDs.
    """
    try:
        start_node = graph.vs.find(id=user_track).index
    except ValueError:
        #raise ValueError(f"Track '{user_track}' not found in the graph.")
        return None

    walk = restart_random_walk_exclude_user_pid(graph, start_node, user_pid, num_steps, restart_prob)

    visit_counts = {}
    for node in walk:
        track_id = graph.vs[node]["id"]
        #if user_pid not in map(int, graph.vs[node]["playlists"].split(",")):
        visit_counts[track_id] = visit_counts.get(track_id, 0) + 1

    recommended_tracks = sorted(visit_counts, key=visit_counts.get, reverse=True)
    eligible_tracks = [track for track in recommended_tracks if track != user_track]
    return eligible_tracks[:top_n]

# Step 4: Accuracy testing function
def test_recommendation_accuracy_restart(graph, test_data_path, num_steps=5, top_n=1, restart_prob=0.15):
    with open(test_data_path, "r") as f:
        test_data = json.load(f)

    playlists = test_data["playlists"]
    input_track_uris_per_playlist = {}
    eligible_input_uri_per_playlist = {}
    for playlist in tqdm(playlists, desc="Processing playlists"):
        this_track_uris = [track["track_uri"] for track in playlist["tracks"]]
        input_track_uris_per_playlist[playlist["pid"]] = this_track_uris
        tracks_in_graph = [track_uri for track_uri in this_track_uris if track_uri in graph.vs["id"]]
        eligible_input_uri_per_playlist[playlist["pid"]] = random.choice(tracks_in_graph) if len(tracks_in_graph) > 0 else None

    # filter key
    testable_playlists = [pid for pid in input_track_uris_per_playlist.keys() if eligible_input_uri_per_playlist[pid] != None]
    recommendation_success = np.zeros(len(testable_playlists))
    recommended_degree = np.zeros(len(testable_playlists))

    for i, pid in enumerate(tqdm(testable_playlists, desc="Testing playlists")):
        input_track_uris = input_track_uris_per_playlist[pid]
        eligible_input_uri = eligible_input_uri_per_playlist[pid]
        recommended_tracks = recommend_tracks_by_pid_restart(graph, pid, eligible_input_uri, num_steps, top_n, restart_prob)
        if recommended_tracks is not None:
            recommended_degrees = [graph.degree(graph.vs.find(id=track).index) for track in recommended_tracks]
            recommended_degree[i] = np.mean(recommended_degrees)
            if any(track in recommended_tracks for track in input_track_uris):
                recommendation_success[i] = 1
        else:
            recommended_degree[i] = 0

    accuracy = np.mean(recommendation_success)
    print(f"Accuracy: {accuracy * 100:.2f}%")

    mean_recommended_degree = np.mean(recommended_degree)
    print(f"Mean recommended degree: {mean_recommended_degree:.2f}")

    # recommendation_scores = recommendation_success / np.log10(recommended_degree)
    recommendation_scores = recommendation_success / recommended_degree
    mean_recommendation_score = np.mean(recommendation_scores)
    print(f"Mean recommendation score: {mean_recommendation_score:.3e}")

    return accuracy


# Step 5: Run the test
test_data_path = "../dataset/data/mpd.slice.999000-999999.json"  # Path to your test JSON file
test_recommendation_accuracy_restart(graph, test_data_path, restart_prob=0.15, num_steps=5, top_n=1)


Processing playlists:   0%|          | 0/1000 [00:00<?, ?it/s]

Testing playlists:   0%|          | 0/987 [00:00<?, ?it/s]

Accuracy: 3.95%
Mean recommended degree: 844.26
Mean recommendation score: 1.827e-04


0.03951367781155015

In [21]:
import random
import json

# Step 1: Load the graph from GraphML
# graph = ig.Graph.Read_GraphML("../network-parser/track_graph.graphml")
# graph.to_undirected()  # Ensure the graph is undirected

# random.seed(44)

# Step 3: Recommendation function
def recommend_tracks_by_highest_degree_neighbors(graph, user_pid, user_track, top_n=1):
    """
    Recommend tracks based on a restart random walk avoiding tracks in the user's playlist.
    
    Args:
        graph (igraph.Graph): The input graph with edge weights.
        user_pid (int): The user's playlist ID to exclude tracks from.
        user_track (str): The track ID of the starting node.
        top_n (int): Number of recommendations to return.
        
    Returns:
        list: A list of recommended track IDs.
    """
    try:
        start_node = graph.vs.find(id=user_track).index
    except ValueError:
        #raise ValueError(f"Track '{user_track}' not found in the graph.")
        return None

    neighbors = graph.neighbors(start_node, mode="all")
    neighbors_sorted = sorted(neighbors, key=lambda x: graph.degree(x), reverse=True)
    top_neighbors = neighbors_sorted[:top_n]

    return [graph.vs[node]["id"] for node in top_neighbors]

# Step 4: Accuracy testing function
def test_recommendation_accuracy_highest_degree_neighbors(graph, test_data_path, top_n=1):
    with open(test_data_path, "r") as f:
        test_data = json.load(f)

    playlists = test_data["playlists"]
    input_track_uris_per_playlist = {}
    eligible_input_uri_per_playlist = {}
    for playlist in tqdm(playlists, desc="Processing playlists"):
        this_track_uris = [track["track_uri"] for track in playlist["tracks"]]
        input_track_uris_per_playlist[playlist["pid"]] = this_track_uris
        tracks_in_graph = [track_uri for track_uri in this_track_uris if track_uri in graph.vs["id"]]
        eligible_input_uri_per_playlist[playlist["pid"]] = random.choice(tracks_in_graph) if len(tracks_in_graph) > 0 else None

    # filter key
    testable_playlists = [pid for pid in input_track_uris_per_playlist.keys() if eligible_input_uri_per_playlist[pid] != None]
    recommendation_success = np.zeros(len(testable_playlists))
    recommended_degree = np.zeros(len(testable_playlists))

    for i, pid in enumerate(tqdm(testable_playlists, desc="Testing playlists")):
        input_track_uris = input_track_uris_per_playlist[pid]
        eligible_input_uri = eligible_input_uri_per_playlist[pid]
        recommended_tracks = recommend_tracks_by_highest_degree_neighbors(graph, pid, eligible_input_uri, top_n)
        if recommended_tracks is not None:
            recommended_degrees = [graph.degree(graph.vs.find(id=track).index) for track in recommended_tracks]
            recommended_degree[i] = np.mean(recommended_degrees)
            if any(track in recommended_tracks for track in input_track_uris):
                recommendation_success[i] = 1
        else:
            recommended_degree[i] = 0

    accuracy = np.mean(recommendation_success)
    print(f"Accuracy: {accuracy * 100:.2f}%")

    mean_recommended_degree = np.mean(recommended_degree)
    print(f"Mean recommended degree: {mean_recommended_degree:.2f}")

    recommendation_scores = recommendation_success / recommended_degree
    # recommendation_scores = recommendation_success / np.log10(recommended_degree)
    mean_recommendation_score = np.mean(recommendation_scores)
    print(f"Mean recommendation score: {mean_recommendation_score:.3e}")

    return accuracy


# Step 5: Run the test
test_data_path = "../dataset/data/mpd.slice.999000-999999.json"  # Path to your test JSON file
test_recommendation_accuracy_highest_degree_neighbors(graph, test_data_path, top_n=1)


Processing playlists:   0%|          | 0/1000 [00:00<?, ?it/s]

Testing playlists:   0%|          | 0/987 [00:00<?, ?it/s]

Accuracy: 6.69%
Mean recommended degree: 3924.42
Mean recommendation score: 2.754e-05


0.0668693009118541

# Community Detection

In [13]:
import igraph as ig

# Step 1: Load the graph and ensure it's undirected
# graph = ig.Graph.Read_GraphML("../network-parser/track_graph.graphml")
# graph.to_undirected()

# Step 2: Perform community detection
communities = graph.community_multilevel()

# Step 3: Compute node degrees
degrees = graph.degree()

# Step 4: Get top nodes in each community
top_nodes_per_community = {}
for i, community in enumerate(communities):
    # Rank nodes by degree within the community
    ranked_nodes = sorted(community, key=lambda node: degrees[node], reverse=True)
    # Get top 5 nodes
    top_nodes_per_community[i] = ranked_nodes[:5]

# Step 5: Print results
print(f"Number of communities detected: {len(communities)}")
for community_id, top_nodes in top_nodes_per_community.items():
    print(f"\nTop nodes in Community {community_id}:")
    for node in top_nodes:
        print(f"  Node {graph.vs[node]['id']} with degree {degrees[node]}")


Number of communities detected: 78

Top nodes in Community 0:
  Node spotify:track:1snNAXmmPXCn0dkF9DaPWw with degree 2340
  Node spotify:track:1sNSG13fsK6KPKKNIQXXrh with degree 2033
  Node spotify:track:0dPBlz8N0veeceejtuvpPz with degree 1949
  Node spotify:track:6ZT3coOj97F6CVvruPtnox with degree 1924
  Node spotify:track:0k93MXOj0kSXo84SvSDeUz with degree 1780

Top nodes in Community 1:
  Node spotify:track:2ZWlPOoWh0626oTaHrnl2a with degree 2056
  Node spotify:track:39KG4kom3enSx4GTThuDGt with degree 2009
  Node spotify:track:4auZhbHhgbiSMeh0kiHCRk with degree 1936
  Node spotify:track:0d28khcov6AiegSCpG5TuT with degree 1837
  Node spotify:track:5QldjuXcxplhjjUqLrzl6H with degree 1756

Top nodes in Community 2:
  Node spotify:track:1tMyqN7bNCokdg7jWgKPc8 with degree 115
  Node spotify:track:1y58wFuYYnVsCnY8DsOELY with degree 115
  Node spotify:track:5ib3EGG06XUnUf7hzDnheL with degree 115
  Node spotify:track:5zLvo9lek82KN9QKRpJojD with degree 82
  Node spotify:track:7tor1N7ix61iCk

# Community based random walk

In [22]:
# specify test data
test_data_path = "../dataset/data/mpd.slice.999000-999999.json"

# community detection
communities = graph.community_multilevel()

# convert communities to induced subgraph
subgraphs = [graph.induced_subgraph(community) for community in communities]

# combine subgraphs using disjoint_union
combined_graph = subgraphs[0]  # Start with first subgraph
for sg in subgraphs[1:]:
    combined_graph = combined_graph.disjoint_union(sg)

# test recommendation accuracy
test_recommendation_accuracy_restart(combined_graph, test_data_path, restart_prob=0.15, num_steps=50, top_n=1)

Processing playlists:   0%|          | 0/1000 [00:00<?, ?it/s]

Testing playlists:   0%|          | 0/987 [00:00<?, ?it/s]

Accuracy: 3.24%
Mean recommended degree: 609.95
Mean recommendation score: 2.985e-04


0.03242147922998987